<a href="https://colab.research.google.com/github/Nandinishyam/Machine_learning/blob/main/SMS_spam_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras


In [1]:
pip install tensorflow --upgrade


In [54]:
from keras.layers import Bidirectional, LSTM, GRU
# Bidirectional allows to read string forwards and backwards for efficiency
# Long term short memory


In [5]:
!pip install keras_tuner
from keras_tuner import HyperModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner.tuners import RandomSearch
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

In [13]:
data = pd.read_csv('spam.csv', encoding='latin1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [75]:

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [72]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [73]:
#data preprocessing
def preprocess(text):
  text=text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stopwords.words('english')])
  return text

In [18]:
data['v2']=data['v2'].apply(preprocess)

In [19]:
label_encoder = LabelEncoder()
data['v1'] = label_encoder.fit_transform(data['v1'])

In [21]:
x = data['v2']
y = data['v1']

In [25]:
max_words=5000
max_len=200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
x_padded = pad_sequences(sequences, maxlen=max_len)



In [26]:
x_train, x_test, y_train, y_test = train_test_split(x_padded, y, test_size=0.2, random_state=42)

In [55]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    Bidirectional(GRU(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [56]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [59]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/100
112/112 ━━━━━━━━━━━━━━━━━━━━ 48s 425ms/step - accuracy: 0.9997 - loss: 0.0026 - val_accuracy: 0.9809 - val_loss: 0.0941
Epoch 2/100
112/112 ━━━━━━━━━━━━━━━━━━━━ 81s 418ms/step - accuracy: 0.9989 - loss: 0.0037 - val_accuracy: 0.9776 - val_loss: 0.1226
Epoch 3/100
112/112 ━━━━━━━━━━━━━━━━━━━━ 81s 413ms/step - accuracy: 0.9999 - loss: 5.1090e-04 - val_accuracy: 0.9798 - val_loss: 0.1266
Epoch 4/100
112/112 ━━━━━━━━━━━━━━━━━━━━ 46s 410ms/step - accuracy: 1.0000 - loss: 3.2732e-04 - val_accuracy: 0.9787 - val_loss: 0.1459


In [60]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.9808 - loss: 0.0841
Test Loss: 0.0938
Test Accuracy: 0.9785


In [61]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(x_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Assuming binary classification
print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))


from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)
print(f'AUC: {auc:.4f}')


35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step
[[956   9]
 [ 15 135]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       965
           1       0.94      0.90      0.92       150

    accuracy                           0.98      1115
   macro avg       0.96      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115

AUC: 0.9858


In [ ]:
user_input = input("Enter your SMS content: ").strip()

if not user_input:
    print("Error: Empty input. Please enter valid SMS content.")
else:
    try:
        # Preprocess the input
        processed_input = preprocess(user_input)

        # Tokenize and pad the input
        sequence = tokenizer.texts_to_sequences([processed_input])
        padded_sequence = pad_sequences(sequence, maxlen=200)  # Ensure max_len matches training

        # Predict the label
        prediction = model.predict(padded_sequence, verbose=0)
        print(prediction)
        label = "spam" if prediction[0][0] > 0.4 else "not spam"

        # Output the result
        print(f"The SMS content is classified as: {label}")

    except Exception as e:
        print(f"An error occurred during processing: {e}")
